In [1]:
import json

In [3]:
import os
cwd = os.getcwd()

In [4]:
cwd

'/home/ubuntu/COVID-19/notebooks'

In [5]:
TEST_JSON = "../data/input/biorxiv_medrxiv/biorxiv_medrxiv/0015023cc06b5362d332b3baf348d11567ca2fbb.json"
TEST_JSON_2 = "../data/biorxiv_medrxiv/biorxiv_medrxiv/00340eea543336d54adda18236424de6a5e91c9d.json"
test_json = json.load(open(TEST_JSON, 'r'))

papers_list = [json.load(open(TEST_JSON, 'r')),
               json.load(open(TEST_JSON_2, 'r')),
               json.load(open(TEST_JSON, 'r')),
               json.load(open(TEST_JSON_2, 'r'))]

FileNotFoundError: [Errno 2] No such file or directory: '../input/data/biorxiv_medrxiv/biorxiv_medrxiv/0015023cc06b5362d332b3baf348d11567ca2fbb.json'

<h1 style="color:red">Knowledge Graph</h1>

# Setup Code

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [3]:
import spacy
import functools
from functools import reduce, partial
import numpy as np
import operator
import requests

In [4]:
import os
import re
import json
import time
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
ENTITY_INDICATORS = ["NOUN", "PROPN"]

## Helper Functions

In [ ]:
def rm_dups(relations):
    """Removes duplicate triples from list."""
    return list(set(relations))

In [ ]:
def regex_filter(text):
    """Removing numbers and puncutation."""
    text = re.sub(r" \d+", "",text)
    return re.sub(r"[^A-Za-z0-9 -]+", "",text)

In [ ]:
def construct_name(author):
    """Combing first and last name of authors."""
    return author["first"].lower() + " " + author["last"].lower()

In [ ]:
def collect_text(text):
    """Combing list of text elements into one."""
    return " ".join(regex_filter(seg['text']) for seg in text)
    

In [ ]:
def collect_bib_text(bib):
    ref_iter = bib.values()
    return " ".join(regex_filter(ref['title']) for ref in ref_iter)

In [ ]:
def get_entities(text):
    """Returns the entities from a spacy token sequence."""
    return [token.text.lower() for token in text if token.pos_ in ENTITY_INDICATORS]

---

In [ ]:
def get_relation(nlp, paper):
    """
    Read paper and collect triple relations.
    """
    
    triples = []
    
    # Paper ID
    paperid = paper['paper_id']
    
    # Author
    authors_list = list(map(construct_name, paper["metadata"]["authors"]))
    authors_triples = [(author, paperid, "has_author") for author in authors_list]
    
    # Title
    title_entities = get_entities(nlp(paper['metadata']['title']))
    title_triples = [(title_entity, paperid, "has_title") for title_entity in title_entities]
    
    # Abtract
    abstract_text = collect_text(paper['abstract'])
    abstract_entities = get_entities(nlp(abstract_text))
    abstract_triples = [(abs_entity, paperid, "has_abstract") for abs_entity in abstract_entities]
    
    # Body
    body_text = collect_text(paper['body_text'])
    body_entities = get_entities(nlp(body_text))
    body_triples = [(body_entity, paperid, "has_body") for body_entity in body_entities]

    # Bibliography
    bib_text = collect_bib_text(paper['bib_entries'])
    bib_entities = get_entities(nlp(bib_text))
    bib_triples = [(bib_entity, paperid, "has_body") for bib_entity in bib_entities]
    
    triples.extend(rm_dups(authors_triples))
    triples.extend(rm_dups(title_triples))
    triples.extend(rm_dups(abstract_triples))
    triples.extend(rm_dups(body_triples))
    triples.extend(rm_dups(bib_triples))
    
    return triples

In [ ]:
def get_mesh_relations(kg_data):
    """
    Fill knowledge graph with supporting references from MeSH
    """
    mesh_url = "https://id.nlm.nih.gov/mesh/lookup/term?label={query}&match=contains&limit=5"

    triples = []
    for idx, row in enumerate(kg_data.iterrows()):
        entity = row[1]['entity']

        req = requests.get(mesh_url.format(query=entity),
                           headers={"content-type": "application/json"}, verify=False)

        req_eval = eval(req.text)
        if len(req_eval) != 0:
            mesh_tup = [(ref['label'].lower(), row[1]['paper_id'], row[1]['relation']) for ref in req_eval]
            triples.append(mesh_tup)

        if idx % 100 == 0:
            print("        {idx}/{size}".format(idx=idx, size=kg_df.shape[0]))

    mesh_df = pd.DataFrame(reduce(operator.concat, triples)) 
    mesh_df.columns = ["entity", "paper_id", "relation"]
    
    return mesh_df

In [ ]:
def create_knowledge_graph(paper_list):
    """Create Knowledge Graph"""
    
    # Construct KG from nlp pipeline
    relations = list(map(functools.partial(get_relation, nlp), papers_list))
    kg_df = pd.DataFrame(reduce(operator.concat, relations))
    kg_df.columns = ["entity", "paper_id", "relation"]
    
    # fill in KG with connects in MESH ontology
    full_knowledge_graph = get_mesh_relations(kg_df)
    
    # combine the two sets of triples
    return pd.concat([kg_df, full_knowledge_graph],
                     axis=0).reset_index(drop=True)
    
    

---

In [ ]:
model = "en_core_web_lg"
nlp = spacy.load(model)

In [ ]:
# creating stopword filter component of nlp
com_wrds_df = pd.read_csv("../data/external_data/common_words.csv", header=0)
for word in com_wrds_df.iterrows():
    nlp.vocab[(word[1]['words'])].is_stop = True

def remove_stopwords(doc):
    """Spacy Component that removes stopwords."""
    return [token for token in doc if not token.is_stop]

nlp.add_pipe(remove_stopwords, name="filter_stopwords", last=True)

In [ ]:
relations = list(map(functools.partial(get_relation, nlp), papers_list))

kg_df = pd.DataFrame(reduce(operator.concat, relations))
kg_df.columns = ["entity", "paper_id", "relation"]

In [ ]:
full_knowledge_graph = get_mesh_relations(kg_df)

In [ ]:
full_knowledge_graph = pd.concat([kg_df, mesh_df],axis=0).reset_index(drop=True)

In [ ]:
full_knowledge_graph.to_csv("sampleresult1.csv", index=False)

<h1 style="color:red">BERT Embeddings</h1>

In [ ]:
!pip install spacy-transformers

In [ ]:
!python -m spacy download en_trf_bertbaseuncased_lg

In [ ]:
!pip install -U spacy[cuda92]

In [1]:
import spacy
import torch
import numpy

In [ ]:
spacy.prefer_gpu()

In [ ]:
if torch.cuda.is_available():
    pass
    #torch.set_default_tensor_type("torch.cuda.FloatTensor")

In [2]:
bert_model = "en_trf_bertbaseuncased_lg"
bert_nlp = spacy.load(bert_model)

OSError: [E050] Can't find model 'en_trf_bertbaseuncased_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [ ]:
def generate_embedddings(paper_list, embed_pipeline):
        
    def get_id_title(paper):
        return (paper['paper_id'],
                paper['metadata']['title'])
    
    def get_embed(emb_mdl, text):
        # Summing gives us sentence level embedding
        # https://colab.research.google.com/github/explosion/spacy-pytorch-transformers/blob/master/examples/Spacy_Transformers_Demo.ipynb
        return (text[0], emb_mdl(text[1]).tensor.sum(axis=0))
        
    # processing embeddings
    title_text = map(get_id_title, papers_list)
    title_embeds = list(map(partial(get_embed, embed_pipeline),
                            title_text))
    
    # converting to dataframe
    embd_df = pd.DataFrame(title_embeds) # convert to dataframe
    embd_df = pd.DataFrame(embd_df[1].values.tolist(), index = embd_df[0]) # split vector into multiple columns
    embd_df.index = embd_df.index.set_names('paper_id')# rename index to paper_id
    
    return embd_df

    



In [ ]:
embd_df = generate_embedddings(papers_list, bert_nlp)

In [ ]:
embd_df.to_csv("sandbox_output/bert_title_embeddings.csv", index=True)

<h1 style="color:red">BERT QA</h1>

In [ ]:
!pip install --user cdqa

In [ ]:
!pip install --upgrade pandas

In [ ]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model, download_bnpp_data

In [ ]:
download_model(model='bert-squad_1.1', dir='./models')

In [ ]:
download_bnpp_data(dir='./data/bnpp_newsroom_v1.1/')

In [ ]:
df = pd.read_csv('./data/bnpp_newsroom_v1.1/bnpp_newsroom-v1.1.csv', converters={'paragraphs': literal_eval})
df = filter_paragraphs(df)
df.head()

In [ ]:
def get_record(paper):
    
    return (paper['paper_id'],
            paper['metadata']['title'],
            collect_text(paper['abstract']),
            [para['text'] for para in paper['body_text']]
    )

In [ ]:
test_Data = pd.DataFrame(list(map(get_record, papers_list)), columns=['paper_id', 'title', 'abstract', 'paragraphs'])
test_Data = filter_paragraphs(test_Data)

In [ ]:
test_Data['paragraphs'][0]

In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=test_Data)

In [ ]:
query = 'What is the largest known picornavirus UTR?'
prediction = cdqa_pipeline.predict(query)

In [ ]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

In [3]:
import spacy
from drqa.tokenizers import spacy_tokenizer

In [4]:
spacy.load('en_core_web_lg')

In [6]:
from spacy.util import get_package_path

In [7]:
get_package_path("en_core_web_lg")

PosixPath('/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/en_core_web_lg')

In [5]:
tok = spacy_tokenizer.SpacyTokenizer()
tok.tokenize('hello world').words()

['hello', 'world']

In [5]:
import drqa.reader

In [8]:
drQA_model_path = "../data/drQa_data/reader/single.mdl"

In [14]:
drqa.reader.set_default('model', drQA_model_path)
reader = drqa.reader.Predictor()

Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 462, in read_nonblocking
    raise TIMEOUT('Timeout exceeded.')
pexpect.exceptions.TIMEOUT: Timeout exceeded.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/ba

pexpect.exceptions.TIMEOUT: Timeout exceeded.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/reader/predictor.py", line 31, in init
    PROCESS_TOK = tokenizer_class(annotators=annotators)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 33, in __init__
    self._launch()
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 61, in _launch
    self.corenlp.expect_exact('NLP>', searchw

  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 107, in expect_loop
    return self.timeout(e)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 70, in timeout
    raise TIMEOUT(msg)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 70, in timeout
    raise TIMEOUT(msg)
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 78892
child_fd: 37
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfi

In [15]:
reader.predict(document="This is a document~", question="Whatt?")

Process ForkPoolWorker-21:
Process ForkPoolWorker-22:
Traceback (most recent call last):
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
Process ForkPoolWorker-23:
Traceback (most recent call last):
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 462, in read_nonblocking
    raise TIMEOUT('Timeout exceeded.')
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
Traceback (most recent call last):
Process ForkPoolWorker-24:
pexpect.exceptions.TIMEOUT: Timeout exceeded.
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 462, in read_nonblocking
    raise TIMEOUT('Timeout exceeded.')
  File "/Users/bajayi-obe/.virtualenv

  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/spawnbase.py", line 390, in expect_exact
    return exp.expect_loop(timeout)
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 79396
child_fd: 37
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 100000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_string:
    0: "b'NLP>'"
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-

  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 61, in _launch
    self.corenlp.expect_exact('NLP>', searchwindowsize=100)
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 79685
child_fd: 37
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 100000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0

  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 33, in __init__
    self._launch()
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 33, in __init__
    self._launch()
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/reader/predictor.py", line 31, in init
    PROCESS_TOK = tokenizer_class(annotators=annotators)
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 61, in _launch
    self.corenlp.expect_exact('NLP>', searchwindowsize=100)
  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 61, in _launch
    self.corenlp.expect_exact('NLP>', searchwind

pexpect.exceptions.TIMEOUT: Timeout exceeded.
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
pexpect.exceptions.TIMEOUT: Timeout exceeded.
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 462, in read_nonblocking
    raise TIMEOUT('Timeout exceeded.')

During handling of the above exception, another exception occurred:


During handling of the above exception, another exception occurred:

  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
pexpect.exceptions.TIMEOUT: Timeout exceeded.
Traceback (most recent call last):
  File "/Users/ba

...

pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
after: <class 'pexpect.exceptions.TIMEOUT'>
match: None
match_index: None
exitstatus: None
flag_eof: False
pid: 28215
child_fd: 37
closed: False
timeout: 60
delimiter: <class 'pexpect.exceptions.EOF'>
logfile: None
logfile_read: None
logfile_send: None
maxread: 100000
ignorecase: False
searchwindowsize: None
delaybeforesend: 0
delayafterclose: 0.1
delayafterterminate: 0.1
searcher: searcher_string:
    0: "b'NLP>'"
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last

  File "/Users/bajayi-obe/Documents/git_dir/DrQA/drqa/tokenizers/corenlp_tokenizer.py", line 61, in _launch
    self.corenlp.expect_exact('NLP>', searchwindowsize=100)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/spawnbase.py", line 390, in expect_exact
    return exp.expect_loop(timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 107, in expect_loop
    return self.timeout(e)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 70, in timeout
    raise TIMEOUT(msg)
pexpect.exceptions.TIMEOUT: Timeout exceeded.
command: /bin/bash
args: ['/bin/bash']
buffer (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
before (last 100 chars): b'\r\nCaused by: java.lang.ClassNotFoundException: edu.stanford.nlp.pipeline.StanfordCoreNLP\r\nbash-3.2$ '
after: <class 'pexpect.excepti

  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/spawnbase.py", line 390, in expect_exact
    return exp.expect_loop(timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/spawnbase.py", line 390, in expect_exact
    return exp.expect_loop(timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 452, in read_nonblocking
    r, w, e = select_ignore_interrupts([self.child_fd], [], [], timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, i

KeyboardInterrupt: 

Process ForkPoolWorker-492:
Traceback (most recent call last):
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/expect.py", line 99, in expect_loop
    incoming = spawn.read_nonblocking(spawn.maxread, timeout)
  File "/Users/bajayi-obe/.virtualenvs/covid_env/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 462, in read_nonblocking
    raise TIMEOUT('Timeout exceeded.')
pexpect.exceptions.TIMEOUT: Timeout exceeded.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/bajayi-obe/.pyenv/versions/3.6.7/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/Users/b